# 文本处理

### 官方处理流程
```sh
python2 ./data/tools.py -m filter -s ./data/News_info_train.txt -t ./clean_data/News_info_train_filter.txt
python2 ./data/tools.py -m filter -s ./data/News_info_unlabel.txt -t ./clean_data/News_info_unlabel_filter.txt
python2 ./data/tools.py -m filter -s ./data/News_info_validate.txt -t ./clean_data/News_info_validate_filter.txt
python2 ./data/tools.py -m filter -s ./data/News_info_label_train_example100.txt -t ./clean_data/News_info_label_train_example100_filter.txt
```

### 自定义处理流程


In [1]:
import jieba
import pandas as pd
from html.parser import HTMLParser

def is_uchar(uchar):
    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
            return True
    """判断一个unicode是否是数字"""
    if uchar >= u'\u0030' and uchar<=u'\u0039':
            return True        
    """判断一个unicode是否是英文字母"""
    if (uchar >= u'\u0041' and uchar<=u'\u005a') or (uchar >= u'\u0061' and uchar<=u'\u007a'):
            return True
    if uchar in ('-',',','，','。','.','<','>','?'):
            return True
    return False

def str_filter(s):
    ret=''
    for i in s:
        ret += i if is_uchar(i) else ' '
    return ret

class MyHTMLParser(HTMLParser):
    def __init__(self):
        self.dom_stack=[]
        self.text=''
        super(MyHTMLParser,self).__init__()
    def handle_starttag(self, tag, attrs):
        if 'br' == tag:
            self.text +=' CONTROL_BREAK '
        elif 'p' == tag:
            self.text +=' CONTROL_NEWP '
        elif 'span' == tag:
            self.text +=' CONTROL_SPAN_START '
        elif 'strong' == tag:
            self.text +=' CONTROL_STRONG_START '
        self.dom_stack.append(tag)

    def handle_endtag(self, tag):
        if 'span' == tag:
            self.text +=' CONTROL_SPAN_END '
        elif 'strong' == tag:
            self.text +=' CONTROL_STRONG_END '
        self.dom_stack.pop()
        
    def current_dom(self):
        return self.dom_stack[-1] if len(self.dom_stack)>0 else 'empty'
    
    def clear(self):
        self.dom_stack=[]
        self.text=''
    
    def handle_data(self, data):
        data=data.strip()
        data=str_filter(data)
        data=' '.join(jieba.cut(data))
        current_dom = self.current_dom()
        if current_dom == 'title':
            self.title = data
        elif current_dom == 'p':
            self.text += ' CONTROL_EMPTYP ' if 0 == len(data) else ' %s '%(data)
        elif current_dom == 'span':
            self.text += ' CONTROL_EMPTYSPAN ' if 0 == len(data) else ' %s '%(data)
        elif current_dom == 'strong':
            self.text += ' CONTROL_EMPTYSTRONG ' if 0 == len(data) else ' %s '%(data)
        elif current_dom == 'html':
            self.text += '  ' if 0 == len(data) else ' CONTROL_NONEMPTYHTMLSTART %s CONTROL_NONEMPTY_HTML_END '%(data)
        elif current_dom == 'body':
            self.text += '  ' if 0 == len(data) else ' CONTROL_NONEMPTY_BODY_START %s CONTROL_NONEMPTY_BODY_END '%(data)
        elif current_dom == 'h1':
            self.text += ' CONTROL_NONEMPTY_H1_BEGIN %s CONTROL_NONEMPTY_H1_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_H1 '
        elif current_dom == 'h2':
            self.text += ' CONTROL_NONEMPTY_H2_BEGIN %s CONTROL_NONEMPTY_H2_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_H2 '
        elif current_dom == 'h3':
            self.text += ' CONTROL_NONEMPTY_H3_BEGIN %s CONTROL_NONEMPTY_H3_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_H3 '
        elif current_dom == 'h4':
            self.text += ' CONTROL_NONEMPTY_H4_BEGIN %s CONTROL_NONEMPTY_H4_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_H4 '
        elif current_dom == 'h5':
            self.text += ' CONTROL_NONEMPTY_H5_BEGIN %s CONTROL_NONEMPTY_H5_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_H5 '
        elif current_dom == 'br':
            self.text += ' CONTROL_NONEMPTY_BR_BEGIN %s CONTROL_NONEMPTY_BR_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_BR '
        elif current_dom == 'tr':
            self.text += ' CONTROL_NONEMPTY_TR_BEGIN %s CONTROL_NONEMPTY_TR_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_TR '
        elif current_dom == 'td':
            self.text += ' CONTROL_NONEMPTY_TD_BEGIN %s CONTROL_NONEMPTY_TD_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_TD '
        elif current_dom == 'th':
            self.text += ' CONTROL_NONEMPTY_TH_BEGIN %s CONTROL_NONEMPTY_TH_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_TH '
        elif current_dom == 'ul':
            self.text += ' CONTROL_NONEMPTY_UL_BEGIN %s CONTROL_NONEMPTY_UL_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_UL '
        elif current_dom == 'li':
            self.text += ' CONTROL_NONEMPTY_LI_BEGIN %s CONTROL_NONEMPTY_LI_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_LI '
        elif current_dom == 'ol':
            self.text += ' CONTROL_NONEMPTY_OL_BEGIN %s CONTROL_NONEMPTY_OL_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_OL '
        elif current_dom == 'blockquote':
            self.text += ' CONTROL_NONEMPTY_BLOCK_QUOTE_BEGIN %s CONTROL_NONEMPTY_BLOCK_QUOTE_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_BLOCK_QUOTE '
        elif current_dom == 'i':
            self.text += ' CONTROL_NONEMPTY_I_BEGIN %s CONTROL_NONEMPTY_I_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_I '
        elif current_dom == 'b':
            self.text += ' CONTROL_NONEMPTY_B_BEGIN %s CONTROL_NONEMPTY_B_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_B '
        elif current_dom == 'how':
            self.text += ' CONTROL_NONEMPTY_HOW_BEGIN %s CONTROL_NONEMPTY_HOW_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_HOW '
        elif current_dom == 'le':
            self.text += ' CONTROL_NONEMPTY_LE_BEGIN %s CONTROL_NONEMPTY_LE_END '%(data) if len(data) > 0 else ' CONTROL_EMPTY_LE '
        elif current_dom in ['img', 'a']:
            self.text += ' CONTROL_UNTRACED_ITEM '
        else:
            print("In dom:%s, Skip string: \"%s\""%(current_dom, data))
    
    def get_text(self):
        return self.text
    
    def get_title(self):
        return self.title

# 数据预处理

- 解析HTML
- 过滤无效字符
- 生成文本
- 拼接数据


In [2]:
news_info_train = pd.read_csv('./data/News_info_train.txt',names=['id', 'html', 'Pics'], sep='\t', index_col='id')
news_pic_label_train=pd.read_csv('./data/News_pic_label_train.txt',names=['id', 'label', 'target_pic', 'target_text'], sep='\t', index_col='id')
news_info_train['title']=''
news_info_train['text']=''
def dom_apply(row):
    parser.clear()
    parser.feed('<html>%s</html>'%str.strip(row['html']))
    row['title']=parser.get_title()
    row['text']=parser.get_text()
    return row
parser = MyHTMLParser()
news_info_train=news_info_train.apply(dom_apply, axis=1)
parser.close()
news_info_train.drop('html',axis=1,inplace=True)
news_info_train['Pics'] = news_info_train['Pics'].fillna('')
news_pic_label_train.loc[:,('target_pic', 'target_text')]=news_pic_label_train.loc[:,('target_pic', 'target_text')].fillna('')
news_pic_label_train['target_text']=news_pic_label_train['target_text'].apply(lambda x:' '.join(jieba.cut(x)))
news_info_train=news_info_train.join(news_pic_label_train)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.688 seconds.
Prefix dict has been built succesfully.


In [3]:
news_info_train.to_hdf('./clean_data/data.hdf5','news_info_train')
news_info_train

,Pics,title,text,label,target_pic,target_text
id,,,,,,
D0000001,P0000001.JPEG;P0000002.JPEG;P0000003.JPEG;P000...,惠尔 新品 冷色 系 实木 多层 地板 系列,CONTROL_NEWP CONTROL_EMPTYP CONTROL_BRE...,0,,
D0000002,P0000016.JPEG;P0000017.JPEG;P0000018.JPEG;P000...,82 周年 庆 收到 八方 来 贺 之 第一波 女士 专场 福利,CONTROL_NEWP CONTROL_EMPTYP CONTROL_BRE...,2,,
D0000003,P0000025.JPEG;P0000026.JPEG;P0000027.JPEG;P000...,"黄河 片区 压轴 大戏 , 美林 康城 铂悦府 1 月 13 日 优惠 升级",CONTROL_NEWP 自 2017 年 CONTROL_STRONG_STAR...,2,,
D0000004,P0000032.JPEG;P0000033.JPEG;P0000034.JPEG;P000...,"早安 世界 上 所有 的 惊喜 与 好运 , 都 是 你 积累 的 人品 与 善良 。",CONTROL_NEWP CONTROL_EMPTYP CONTROL_STRON...,0,,
D0000005,,中文 到底 有 多难 ? 为教 马克 龙说 这句 话 翻译 差点 崩溃,CONTROL_NONEMPTYHTMLSTART 中文 到底 有 多难 为教 马克 ...,0,,
D0000006,P0000039.JPEG;P0000040.JPEG,空调 送风 千篇一律 卡萨帝 CES 展示 1 台 空调 吹 多种 温度,CONTROL_NEWP 智慧 家庭 将 成为 2018 家居 家电 领域 新风 口...,0,,
D0000007,,手机 辐射 美女 竟然 这么 用,CONTROL_NONEMPTYHTMLSTART 手机 辐射 美女 竟然 这么 用 CO...,0,,
D0000008,P0000041.JPEG;P0000042.JPEG;P0000043.JPEG;P000...,68 岁 老人 走路 送 快递 7 年 为 还 老伴 治病 欠 的 80 万,CONTROL_NEWP 来源 看看 新闻 CONTROL_BREAK ...,0,,
D0000009,,师哥 送 小妖女 情人 泪,CONTROL_NONEMPTYHTMLSTART 艳骨 ， 师哥 送 小妖女 情人 泪 ...,0,,


### 生成语料库

从label和unlabel的数据中抽取所有文本

In [6]:
news_info_unlabeled = pd.read_csv('./data/News_info_unlabel.txt',names=['id', 'html', 'Pics'], sep='\t', index_col='id')
news_info_unlabeled['title']=''
news_info_unlabeled['text']=''
def dom_apply(row):
    parser.clear()
    parser.feed('<html>%s</html>'%str.strip(row['html']))
    row['title']=parser.get_title()
    row['text']=parser.get_text()
    return row
parser = MyHTMLParser()
news_info_unlabeled=news_info_unlabeled.apply(dom_apply, axis=1)
parser.close()
news_info_unlabeled.drop('html',axis=1,inplace=True)
news_info_unlabeled['Pics'] = news_info_unlabeled['Pics'].fillna('')

TypeError: ('%d format: a number is required, not str', 'occurred at index D0081936')